In [43]:
import pandas as pd

In [44]:
# Carregar dataset
df = pd.read_csv('../datasets/dataset-case_study.csv')

# Informações sobre o conjunto de dados
print(df.shape)
print(df.columns)

# Contar quantas linhas tem para cada plataforma
platform_counts = df['platform'].value_counts()
print("\nDistribuição por plataforma:", platform_counts)

(5933, 9)
Index(['restaurante_name', 'user_name', 'review_date', 'stars', 'comment_text',
       'year', 'platform', 'language', 'comment_text_translated'],
      dtype='object')

Distribuição por plataforma: platform
Google             3917
Facebook            847
Foursquare          834
Yelp                310
Zomato               24
Restaurant Guru       1
Name: count, dtype: int64


In [45]:
# Substituir espaços múltiplos por um único espaço e realizar strip
df['comment_text_translated'] = df['comment_text_translated'].astype(str).apply(lambda x: ' '.join(x.split()))

# Remover ocorrências de erros
replacements = {
    'Comida': 'comida',
    'Serviço': 'serviço',
    'Atmosfera': 'atmosfera',
    'Food': 'food',
    'Service': 'service',
    'Atmosphere': 'atmosphere'
}

for uppercase, lowercase in replacements.items():
    df['comment_text_translated'] = df['comment_text_translated'].str.replace(uppercase, lowercase, regex=False)

replacements = [
    'comida: 5', 'food: 5', 'comida: 4', 'food: 4', 'comida: 3', 'food: 3', 'comida: 2', 'food: 2', 'comida: 1', 'food: 1',
    'serviço: 5', 'service: 5', 'serviço: 4', 'service: 4', 'serviço: 3', 'service: 3', 'serviço: 2', 'service: 2', 'serviço: 1', 'service: 1',
    'atmosfera: 5', 'atmosphere: 5', 'atmosfera: 4', 'atmosphere: 4', 'atmosfera: 3', 'atmosphere: 3', 'atmosfera: 2', 'atmosphere: 2', 'atmosfera: 1', 'atmosphere: 1',
]

for r in replacements:
    df['comment_text_translated'] = df['comment_text_translated'].str.replace(r, '')

# Substituir espaços múltiplos por um único espaço e realizar strip
df['comment_text_translated'] = df['comment_text_translated'].astype(str).apply(lambda x: ' '.join(x.split()))

# Remover sentenças nulas
def remove_empty_text_rows(df):
    empty_sentenca_rows = df['comment_text_translated'].isnull() | (df['comment_text_translated'] == '')
    df = df[~empty_sentenca_rows]
    df.reset_index(drop=True, inplace=True)
    return df
df = remove_empty_text_rows(df)

# Remover sentenças duplicadas
comentarios_duplicados = df.duplicated(subset="comment_text_translated").sum()
print("\nTotal de comentários duplicados:", comentarios_duplicados)
print('Removendo comentários duplicados...\n')
df = df.drop_duplicates(subset="comment_text_translated", keep='first').reset_index(drop=True)

# Informações sobre o conjunto de dados
print('Dataset após limpeza:')
print(df.shape)


Total de comentários duplicados: 581
Removendo comentários duplicados...

Dataset após limpeza:
(4973, 9)


In [46]:
# Contar quantas linhas tem para cada plataforma
platform_counts = df['platform'].value_counts()
print("\nDistribuição por plataforma:", platform_counts)


Distribuição por plataforma: platform
Google             3224
Foursquare          731
Facebook            710
Yelp                285
Zomato               22
Restaurant Guru       1
Name: count, dtype: int64


In [47]:
df.to_csv(f'../datasets/dataset-case_study-tratado.csv', index=False)